# Uso de LLM para tareas zero-shot

En este notebook de Python vamos a ver algunos ejemplos de cómo utilizar la API de los Large Language Models (LLM) como GPT de OpenAI para realizar algunas tareas típicas de NLP.

La idea general es construir los prompts adecuados, que nos permitan que el modelo responda resolviendo una tarea típica: resumen, clasificación, Name Entity Recognition. Por supuesto, esto se puede extender a muchas otras tareas. **Esta es la magia del zero-shot learning**

Se divide en dos experimentos:

- Uso de modelos Open Source con la librería transformers
- Uso de modeos propietarios: GPT de OpenAI

Este notebook es una adaptación de otro similar de mi compañero Victor Gallego [@vicgalle](https://github.com/vicgalle).


## Modelos open-source

Podemos hacer uso de modelos Open-source en lugar de la API de OpenAI.
Vamos a hacer un par de ejemplos sencillos de clasificación:

- sentiment analysis para reviews de películas.
- detectar si un empleado está contento o no a través de un comentario

Para ello vamos a utilizar la librería transformers de http://huggingface.co


In [1]:
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [2]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def sentiment(sample):
    # build the prompt
    input_text = f"Review: {sample}\n\n Is this movie review positive? Answer yes or no. Response:"
    x = tokenizer([input_text], return_tensors="pt").input_ids
    outputs = model.generate(
        x, return_dict_in_generate=True, output_scores=True, max_new_tokens=1
    )
    # extract the logits of the yes/no
    logit_yes = outputs.scores[0][:, 2163].cpu().numpy()[0]
    logit_no = outputs.scores[0][:, 465].cpu().numpy()[0]

    prob_yes = np.exp(logit_yes) / (np.exp(logit_yes) + np.exp(logit_no))
    prob_no = 1 - prob_yes

    return {"Probabilidad Positiva": prob_yes, "Probabilidad Negativa": prob_no}

In [4]:
sample = """Nos encontramos, sin duda, ante un espectáculo visual sin precedentes.
Impresionante y romántica historia sobre la tragedia del mítico trasatlántico"""
sentiment(sample)

{'Probabilidad Positiva': 0.95879406,
 'Probabilidad Negativa': 0.04120594263076782}

In [5]:
sample = """Se hunde bajo el peso de su propia ambición"""
sentiment(sample)

{'Probabilidad Positiva': 0.3348655,
 'Probabilidad Negativa': 0.6651344895362854}

In [6]:
sample = "Brando hizo de Don Vito algo que rara vez vemos en las películas: un villano-héroe tragicómico."
sentiment(sample)

{'Probabilidad Positiva': 0.784351,
 'Probabilidad Negativa': 0.21564900875091553}

In [7]:
sample = "Coppola inventa una nueva mirada para el cine y amplía los horizontes de una industria que pedía a gritos savia nueva."
sentiment(sample)

{'Probabilidad Positiva': 0.9453907,
 'Probabilidad Negativa': 0.05460929870605469}

### Segundo ejemplo: están nuestros empleados contentos?

Sólo tenemos que adaptar el prompt!


In [8]:
def employee(sample):
    # build the prompt
    input_text = f"Message of the employee: {sample}\n\n Is this employee happy? Answer yes or no. Response:"
    x = tokenizer([input_text], return_tensors="pt").input_ids
    outputs = model.generate(
        x, return_dict_in_generate=True, output_scores=True, max_new_tokens=1
    )
    # extract the logits of the yes/no
    logit_yes = outputs.scores[0][:, 2163].cpu().numpy()[0]
    logit_no = outputs.scores[0][:, 465].cpu().numpy()[0]

    prob_yes = np.exp(logit_yes) / (np.exp(logit_yes) + np.exp(logit_no))
    prob_no = 1 - prob_yes

    return {"Probabilidad Positiva": prob_yes, "Probabilidad Negativa": prob_no}

In [9]:
sample = """No es asumible que después de antas horas de trabajo no cumplamos el compromiso con el cliente."""
employee(sample)

{'Probabilidad Positiva': 0.38008258,
 'Probabilidad Negativa': 0.6199174225330353}

In [10]:
sample = (
    """Estamos trabajando muchas horas de más, pero merece la pena por lo aprendido."""
)
employee(sample)

{'Probabilidad Positiva': 0.7631787,
 'Probabilidad Negativa': 0.23682129383087158}

In [11]:
sample = """Estamos trabajando muchas horas de más, hay bastante desgaste."""
employee(sample)

{'Probabilidad Positiva': 0.17839426,
 'Probabilidad Negativa': 0.8216057419776917}

## Cómo utilizar la API de OpenAI para realizar tareas típicas de NLP

Lo primero de todo, es cargar la librería de OpenAI para interactuar con los modelos vía API en vez de en el navegador. Puede ser instalada ejecutando `!pip install openai` en una celda del notebook.

Disclaimer: Para utilizar esta sección, se necesita tener una API KEY de OpenAI, que es una feature de pago.


In [12]:
#%pip install openai==0.28

In [13]:
import openai

En un archivo con el siguiente nombre hemos guardado una clave de la API para poder hacer llamadas a los modelos


In [14]:
openai.api_key_path = "../../openai_api_key.txt"

Como ejemplo, vamos a utilizar un texto proveniente de la transcripción de un programa de radio acerca del teletrabajo. El podcast dura unos 15 minutos, por lo que es un texto bastante largo...


In [15]:
texto = """Ser Podcast. Los números nos dicen que de momento no se está quedando pese a que la mayoría de los trabajadores preferirían trabajar en remoto al menos parcialmente. Punto medio, sí. Unos días ir a la pandemia se está desinflando como un globo. Ya sabíamos que en ese momento, el del coronavirus, el teletrabajo fue una obligación, pero también decían los expertos y las empresas que gran parte de ese teletrabajo se podría mantener y que España podría dejar de estar por detrás de otros países en el uso del trabajo a distancia. Pasados dos años, los datos nos dicen que no, que eso fueron almacenamiento para la industria alimentaria. Por tanto, su trabajo es sentado en una oficina atendiendo al teléfono o al email. Una pena porque en la pandemia, Nacho trabajó durante un año todos los días desde casa. Y sí se vio mejorada su vida, tiene tres hijos y la conciliación era mucho mejor. Como Nacho, según un estudio de ADECO, la mitad de las personas que teletrabajaban. Ahora la cifra se ha reducido en un millón que han vuelto a las oficinas. Fue cuando Ana decidió cambiar. Modelo híbrido como el que buscó también Lourdes. Conciliación, ahorro, medio ambiente, para cuidar nuestros entornos, al final evitar desplazamientos innecesarios hace que haya menos tráfico. Coger el coche a diario para Aitor se dedica al comercio online, teletrabaja el 100% de su jornada. Su vida ha su domicilio el año pasado. Apenas el 8% tuvo modelo mixto y solo un 6% teletrabajaban en España algún día a la semana. Con la llegada del coronavirus, esa cifra subió a prácticamente cuatro millones. La cifra bajó el año pasado ya a algo menos de tres millones. Y ahora mismo son dos y medio los que trabajaban en remoto, es decir, hemos pasado de cuatro son coronavirus han vuelto a la presencialidad obligatoria. Es verdad que el teletrabajo en ese momento con la pandemia fue una obligación, que había pocas empresas que estaban preparadas, pero también del cambio de paradigma en la metodología productiva de nuestro país. Lo va a cambiar todo. Va a cambiar la estructura negocial en los convenios colectivos, pero va a cambiar también el modus operandi del desarrollo de nuestros trabajos. entre otras cosas, a que la empresa tuviera que asumir que los medios con los que tiene que teletrabajar los pagan las empresas, que tiene un sistema de control de formeras. Pero como estamos viendo, los datos a día de hoy demuestran que hay marcha atrás. De hecho, si desagregamos aún más los datos, estamos viendo que solo en Madrid y algo en Barcelona hay teletrabajo. En Madrid, por ejemplo, uno de cada cinco trabajadores tiene la opción de hacer trabajo en remoto al menos parcialmente. Es una cifra parecida a la media europea. En Barcelona se acerca al 15% el número de personas que pueden teletrabajar. Todas las demás comunidades la siguen muy de lejos y, por ejemplo, en Murcia, en Castilla-La Mancha o en La Rioja no llegan ni al 10% los que tienen la opción de teletrabajar. Con todo esto, lo que tenemos sobre la mesa es que España, que ya iba retrasada respecto a otros países europeos en esta cuestión, está a día de hoy aún más por detrás. María Benítez, ¿qué tal? Buenas noches. Buenas noches, Jorge. Vamos a repasar los datos. ¿En qué posición está España en cuanto al teletrabajo en comparación con el resto de países europeos? Pues claramente por detrás. Antes de la pandemia, España ocupaba el puesto 14 en el ranking de teletrabajo y ahora ha bajado dos puestos hasta el 16º lugar, claramente por debajo de la media europea ¿por qué? Nuestros compañeros de Radio Madrid le han preguntado estos días a empresarios y a trabajadores. Francisco Aranda, de la patronal madrileña CEIM, apuntaba que es una cuestión de productividad. Tiene el teletrabajo cosas muy positivas, sin duda, como que reduce los tiempos de desplazamiento o que favorece un mayor grado de concentración, pero hay que tener cuidado porque también conlleva situaciones de aislamiento social y profesional que pueden repercutir negativamente en la productividad del empleado. Algunos empresarios también apuntan a que al tener que pagar los costes del teletrabajo a sus empleados, les sale más barato que no teletrabajen, porque así se ahorran este gasto extra. Pero los sindicatos dicen que esto es una excusa. José Varela, responsable de digitalización en el trabajo de UGT, lo decía así. y ya en aquel se afirmaba que todos los costes del teletrabajo tenían que pagarlos el empresario. Estoy hablando de un acuerdo europeo que firmaron las principales patronales europeas. Esto no es un problema de costes, es un problema de fidelizar a los empleados y de tenerlos contentos. Si una empresa decide que no da el teletrabajo por pagar un ordenador y unos pequeños costes de teletrabajo es porque la empresa directamente no es viable. Los costes al final no son más que una excusa. Javier Blasco es director de Diadeco Group Institute. ¿Qué tal? Buenas noches. ¿Qué tal? Buenas noches. Estábamos viendo, según los datos que han publicado ustedes en el monitor que acaban de sacar, que la mitad de los que teletrabajaron durante la pandemia ya no teletrabaja nunca. Esto demuestra que el teletrabajo en muchos casos fue unaimos una norma no sé si anticipada en el tiempo pero parece que en algunas organizaciones tanto de la empresa privada como de la organización pública pues no ha acabado de cuajar y a la pospandemia, algunas de estas han decidido apostar por la perfeccionalidad. No, yo creo que la situación está siendo distinta. Es decir, tanto la situación de teletrabajo en España ya estaba por debajo de la media de la Unión Europea antes de la pandemia y la caída del teletrabajo en la post-pandemia ha sido mayor en el caso español que en el caso del resto de países. Y vemos efectivamente que algunos países, el porcentaje incluso que tenían antes de la pandemia ha mejorado sensiblemente, como en el caso de Portugal, o el caso de Francia. Es decir, que en este año se mueven en cifras por encima del 20% mientras que en el caso español estamos por debajo de la medida y por el mismo nivel que países como similar como Gría, Croacia, Rumanía, Bulgaria, que hay niveles muy distintos de teletrabajo en función de las comunidades autónomas, que básicamente es Madrid la que concentra prácticamente todo el teletrabajo y algo Cataluña, en el resto no. Esto obedece a unas cuestiones entendibles por concentración de las empresas en estas dos comunidades o hay algo detrás de este dato? de empresas, sobre todo de sedes de empresas del sector terciario, para que lo entendamos. Es decir, la zona metropolitana de Barcelona, en el caso de Cataluña y la Cuneo-Madrid con carácter general, es verdad que tienen mayor porcentaje de puestos o de ocupaciones teletrabajables, mientras que otras que, sin embargo, en otros parámetros, ya sea salario, están muy por encima de la media, como el caso de Navarra. Sin embargo, precisamente por ese mayor peso del sector industrial, como ocurre con Cantabria, como ocurre también con Aragón, pues tienen menor porcentaje o doscienes de teletrabajo. Con lo cual, tamaños de empresa, sector de actividad, marca sin duda la mayor presencia del teletrabajo. Bueno, yo creo que ha habido un cúmulo de factores, yo creo que probablemente la normativa no vino en el mejor momento, la normativa de teletrabajo se forzó, se precipitó durante un momento todavía pandémico, yo creo que además lo tuvo probablemente todo el consenso social necesario y probablemente también se vendió como una amenaza. Al final muchas organizaciones, y también probablemente el que los departamentos de recursos humanos no siempre tienen todavía en algunas ocupaciones las métricas suficientes para poder tener un modelo que sea transparente, que publica la productividad, lo del rendimiento y del perform, y esto puede que sea lo que normalmente subyace a esta vuelta a la presencialidad. Que luego haya personas que puedan estar a favor o en contra, va a depender de las situaciones personales, de los espacios de trabajo, de las opciones que tenga la persona, del turista, del serenito de equipos, de las cargas familiares, de los espacios domésticos, para que puedan de alguna u otra manera apostar o no por el tema de trabajo y evidentemente depende del tipo de actividad profesional. Creo que son más las tareas las que en un caso u otro aconsejan presencialidad o trabajan remoto para moverlos a entornos ideales de productividad, pero es verdad que al final las decisiones las toman las personas y las empresas. ya sea de forma parcial o de forma total. ¿Esto ya no es así o sí en el teletrabajo y dependiendo del tipo de profesión, es verdad que en algunos casos se está cometiendo un elemento crítico de animación, al final eso que se llama el salario emocional, o sea, aparte de lo que una persona busca en un puesto de trabajo tiene que ver también con la flexibilidad, la flexibilidad en tiempo de trabajo, la flexibilidad en utilidad también del lugar de trabajo, en este caso el teletrabajo. Gracias presentado también como una vía para favorecer la conciliación, una vía que están avalando incluso los tribunales. Ya hay varias sentencias en este sentido y una de las últimas ha sido por el caso de Verónica. de la cual ha sido favorable y gracias a ellos ahora me permite atender a mis obligaciones familiares. El caso lo ha impulsado el sindicato Uso. Maror Doñez es la responsable de la Federación de Servicios de este sindicato en Galicia. Y se permite a una trabajadora del Contact Center teletrabajar para poder conciliar su vida laboral y familiar. Es un paso más a favor de los trabajadores ver cómo al teletrabajo por pandemia ahora ya trabajan solo exclusivamente desde sus oficinas. Lo de que el teletrabajo llegios y contenidos adicionales, en la app de Cadena Ser y en nuestros canales de Apple Podcast, Spotify, iVoox, Google Podcast y Youtube."""

In [16]:
texto

'Ser Podcast. Los números nos dicen que de momento no se está quedando pese a que la mayoría de los trabajadores preferirían trabajar en remoto al menos parcialmente. Punto medio, sí. Unos días ir a la pandemia se está desinflando como un globo. Ya sabíamos que en ese momento, el del coronavirus, el teletrabajo fue una obligación, pero también decían los expertos y las empresas que gran parte de ese teletrabajo se podría mantener y que España podría dejar de estar por detrás de otros países en el uso del trabajo a distancia. Pasados dos años, los datos nos dicen que no, que eso fueron almacenamiento para la industria alimentaria. Por tanto, su trabajo es sentado en una oficina atendiendo al teléfono o al email. Una pena porque en la pandemia, Nacho trabajó durante un año todos los días desde casa. Y sí se vio mejorada su vida, tiene tres hijos y la conciliación era mucho mejor. Como Nacho, según un estudio de ADECO, la mitad de las personas que teletrabajaban. Ahora la cifra se ha redu

### Ejemplo de clasificación de texto

Lo primero de todo es diseñar el prompt. En él incluimos las instrucciones de la tarea, así como el texto anterior.

El prompt lo escribiremos en dos partes: en esta primera describimos el tipo de tarea que queremos realizar. En este caso, clasificación en unas cuantas categorías:


In [17]:
tarea = """Dados los siguientes temas: política, ciencia, economía, internacional, salud, deporte.
¿En cuáles clasificarías el texto? Responde con una única palabra de las anteriores."""

La siguiente parte añade contexto, concatena el texto y finalmente incluye la descripción de la tarea anterior


In [18]:
prompt = f"""Eres un asistente virtual que responde preguntas acerca del texto de un podcast, devolviendo únicamente la respuesta.
        Este es el texto del podcast:{texto}

        En base a la información del texto anterior, responde a la siguiente pregunta:
        {tarea}"""

A continuación, definimos una función que llama al modelo GPT correspondiente, y completa el texto a partir del prompt:


In [19]:
def completa_prompt(prompt, model="gpt-3.5-turbo"):

    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model, max_tokens=256, temperature=0.3, messages=messages
    )
    # Nos quedamos con texto generado:
    response_text = response["choices"][0]["message"]["content"]
    return response_text

In [20]:
completa_prompt(prompt)

'economía'

### Ejemplo de resumización

Como antes, ahora el único cambio que hay que hacer es describir la nueva tarea


In [21]:
tarea = """Resume la idea principal del texto en una sola frase, para que quepa en un tweet."""

In [22]:
prompt = f"""Eres un asistente virtual que responde preguntas acerca del texto de un podcast, devolviendo únicamente la respuesta.
        Este es el texto del podcast:{texto}

        En base a la información del texto anterior, responde a la siguiente pregunta:
        {tarea}"""

In [23]:
completa_prompt(prompt)

'España se encuentra rezagada en teletrabajo en comparación con otros países europeos, y la pandemia ha provocado una disminución en la cantidad de personas que teletrabajan.'

Vamos a cambiar de modelo


In [24]:
completa_prompt(prompt, model="gpt-4")

'El podcast discute cómo, a pesar de las expectativas iniciales, el teletrabajo en España no se ha mantenido tras la pandemia, con la mayoría de los trabajadores volviendo a la presencialidad, dejando a España por detrás de otros países europeos en términos de trabajo remoto.'

Ahora con unos resúmenes algo más largos...


In [25]:
tarea = """Extrae las tres ideas principales del texto a modo de resumen en bullet points."""

In [26]:
prompt = f"""Eres un asistente virtual que responde preguntas acerca del texto de un podcast, devolviendo únicamente la respuesta.
        Este es el texto del podcast:{texto}

        En base a la información del texto anterior, responde a la siguiente pregunta:
        {tarea}"""

In [27]:
resumen = completa_prompt(prompt)

In [28]:
print(resumen)

- La mayoría de los trabajadores preferirían trabajar en remoto al menos parcialmente, pero los datos muestran que el teletrabajo ha disminuido en España.
- El teletrabajo durante la pandemia mejoró la conciliación familiar y redujo los desplazamientos innecesarios.
- España está por detrás de otros países europeos en cuanto al teletrabajo, ocupando el puesto 16 en el ranking.


### Ejemplo de extracción de información


In [29]:
tarea = """Extrae un json con todas las personas mencionadas con nombre y apellido, indicando como valor la organización a la que pertenecen."""

In [30]:
prompt = f"""Eres un asistente virtual que responde preguntas acerca del texto de un podcast, devolviendo únicamente la respuesta.
        Este es el texto del podcast:{texto}

        En base a la información del texto anterior, responde a la siguiente pregunta:
        {tarea}"""

In [31]:
output = completa_prompt(prompt, model="gpt-4")

In [32]:
print(output)

{
"Nacho": "Industria Alimentaria",
"Ana": "No especificado",
"Lourdes": "No especificado",
"Aitor": "Comercio Online",
"María Benítez": "No especificado",
"Jorge": "No especificado",
"Francisco Aranda": "Patronal Madrileña CEIM",
"José Varela": "UGT",
"Javier Blasco": "Diadeco Group Institute",
"Verónica": "No especificado",
"Maror Doñez": "Federación de Servicios del sindicato Uso"
}
